# **0.0 下準備**
## **0.1 ライブラリのインポート**

In [130]:
# 必要なライブラリのインポート

import pandas as pd
from pandas_datareader import data as web
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from scipy.optimize import minimize
import cvxpy as cp

### **0.1.1 Tickerリスト作成**

In [131]:
# ターゲットとなる企業のティッカーと企業名のリスト
tickers = [
    "1925", "1928", "2413", "2502", "2503", "2802", "2914", "3382", "3402", "3407", "4063", "4188", "4452", "4502", "4503", "4507", "4519", "4523", "4528", "4543", "4568", "4578", "4661", "4689", "4901", "4911", "5020", "5108", "5401", "5713", "5802", "6098", "6178", "6273", "6301", "6326", "6367", "6501", "6502", "6503", "6586", "6594", "6645", "6702", "6752", "6758", "6861", "6869", "6902", "6920", "6954", "6971", "6981", "7011", "7201", "7203", "7267", "7269", "7270", "7309", "7733", "7741", "7751", "7832", "7974", "8001", "8002", "8031", "8035", "8053", "8058", "8113", "8267", "8306", "8308", "8309", "8316", "8411", "8591", "8604", "8630", "8697", "8725", "8750", "8766", "8801", "8802", "8830", "9020", "9021", "9022", "9101", "9202", "9432", "9433", "9434", "9735", "9843", "9983", "9984", "TOPIX100", '1475'
]
# TOPIX100の銘柄のティッカーシンボルリストへの変換
tickers = [tickers + '.T' for tickers in tickers]
print(tickers)

['1925.T', '1928.T', '2413.T', '2502.T', '2503.T', '2802.T', '2914.T', '3382.T', '3402.T', '3407.T', '4063.T', '4188.T', '4452.T', '4502.T', '4503.T', '4507.T', '4519.T', '4523.T', '4528.T', '4543.T', '4568.T', '4578.T', '4661.T', '4689.T', '4901.T', '4911.T', '5020.T', '5108.T', '5401.T', '5713.T', '5802.T', '6098.T', '6178.T', '6273.T', '6301.T', '6326.T', '6367.T', '6501.T', '6502.T', '6503.T', '6586.T', '6594.T', '6645.T', '6702.T', '6752.T', '6758.T', '6861.T', '6869.T', '6902.T', '6920.T', '6954.T', '6971.T', '6981.T', '7011.T', '7201.T', '7203.T', '7267.T', '7269.T', '7270.T', '7309.T', '7733.T', '7741.T', '7751.T', '7832.T', '7974.T', '8001.T', '8002.T', '8031.T', '8035.T', '8053.T', '8058.T', '8113.T', '8267.T', '8306.T', '8308.T', '8309.T', '8316.T', '8411.T', '8591.T', '8604.T', '8630.T', '8697.T', '8725.T', '8750.T', '8766.T', '8801.T', '8802.T', '8830.T', '9020.T', '9021.T', '9022.T', '9101.T', '9202.T', '9432.T', '9433.T', '9434.T', '9735.T', '9843.T', '9983.T', '9984.T',

### **0.1.2 企業名リスト**

In [132]:
def get_company_names(tickers):
    company_names = []
    
    for ticker in tickers:
        try:
            company_info = yf.Ticker(ticker)
            company_names.append(company_info.info["shortName"])
        except:
            company_names.append(None)  # エラーが発生した場合は None をリストに追加
    
    return company_names

company_names = get_company_names(tickers)
print(company_names)

['DAIWA HOUSE INDUSTRY CO', 'SEKISUI HOUSE', 'M3 INC', 'ASAHI GROUP HLDGS', 'KIRIN HOLDINGS COMPANY LIMITED', 'AJINOMOTO CO INC', 'JAPAN TOBACCO INC', 'SEVEN & I HOLDINGS CO LTD', 'TORAY INDUSTRIES INC', 'ASAHI KASEI CORP', 'SHIN-ETSU CHEMICAL CO', 'MITSUBISHI CHEMICAL GROUP CORP', 'KAO CORP', 'TAKEDA PHARMACEUTICAL CO LTD', 'ASTELLAS PHARMA', 'SHIONOGI & CO', 'CHUGAI PHARMACEUTICAL CO', 'EISAI CO', 'ONO PHARMACEUTICAL CO', 'TERUMO CORP', 'DAIICHI SANKYO COMPANY LIMITED', 'OTSUKA HLDGS CO LTD', 'ORIENTAL LAND CO', 'LY CORPORATION', 'FUJIFILM HOLDINGS CORPORATION', 'SHISEIDO COMPANY LIMITED', 'ENEOS HOLDINGS INC', 'BRIDGESTONE CORP', 'NIPPON STEEL CORPORATION', 'SUMITOMO METAL MINING CO', 'SUMITOMO ELECTRIC INDUSTRIES', 'RECRUIT HOLDINGS CO LTD', 'JAPAN POST HLDGS CO LTD', 'SMC CORP', 'KOMATSU', 'KUBOTA CORP', 'DAIKIN INDUSTRIES', 'HITACHI', 'TOSHIBA CORP', 'MITSUBISHI ELECTRIC CORP', 'MAKITA CORP', 'NIDEC CORPORATION', 'OMRON CORP', 'FUJITSU', 'PANASONIC HOLDINGS CORP', 'SONY GROUP COR

### **0.1.3 Ticker-企業名の辞書作成**

In [133]:
def get_ticker_to_name_mapping(tickers):
    ticker_to_name = {}
    
    for ticker in tickers:
        try:
            company_info = yf.Ticker(ticker)
            company_name = company_info.info["shortName"]
            ticker_to_name[ticker] = company_name
        except:
            # エラーが発生した場合は None としてマッピングに追加
            ticker_to_name[ticker] = None
    
    return ticker_to_name

ticker_to_name = get_ticker_to_name_mapping(tickers)
print(ticker_to_name)

## **0.2 株価データセットの作成**

### **0.2.1 月次株価データの作成**

In [ ]:
# 定数の設定
YEARS_BACK = 10  # 取得するデータの期間を10年とする
INTERVAL = "1mo"  # 取得するデータの間隔を月次とする。変更可能な値は"1mo", "1wk", "1d"

In [ ]:
# 期間の計算
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=YEARS_BACK)

# 結果を格納するDataFrameの初期化
result_df = pd.DataFrame()

for ticker in tickers:
    try:
        # データの取得
        data = yf.download(ticker, start=start_date, end=end_date, interval=INTERVAL)
        
        # データの終値を結果のDataFrameに追加
        result_df[ticker] = data["Close"]
    except Exception as e:
        print(f"Error fetching data for {ticker}. Error: {e}")

# Dateを1列目に設定
result_df.reset_index(inplace=True)

# CSVに保存
csv_filename = f"/Users/klynoaguilar/Desktop/01_school/Zemi/23_10_20_weeek-02/topix100_{INTERVAL}_data.csv"
result_df.to_csv(csv_filename, index=False)

print("Data fetching and saving completed!" + ticker)


### **0.2.2 週次株価データの作成**

In [ ]:
# 定数の設定
YEARS_BACK = 3  # 取得するデータの期間を10年とする
INTERVAL = "1wk"  # 取得するデータの間隔を月次とする。変更可能な値は"1mo", "1wk", "1d"

In [ ]:
# 期間の計算
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=YEARS_BACK)

# 結果を格納するDataFrameの初期化
result_df = pd.DataFrame()

for ticker in tickers:
    try:
        # データの取得
        data = yf.download(ticker, start=start_date, end=end_date, interval=INTERVAL)
        
        # データの終値を結果のDataFrameに追加
        result_df[ticker] = data["Close"]
    except Exception as e:
        print(f"Error fetching data for {ticker}. Error: {e}")

# Dateを1列目に設定
result_df.reset_index(inplace=True)

# CSVに保存
csv_filename = f"/Users/klynoaguilar/Desktop/01_school/Zemi/23_10_20_weeek-02/topix100_{INTERVAL}_data.csv"
result_df.to_csv(csv_filename, index=False)

print("Data fetching and saving completed!" + ticker)

### **0.2.3 データの確認と読み込み**

In [ ]:
all_df_monthly = pd.read_csv("./topix100_monthly_data.csv")
all_df_weekly = pd.read_csv('./topix100_weekly_data.csv')

#### **0.2.3.1 データ表示範囲の変更**

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### **0.2.3.2 月次データ確認**

In [ ]:
all_df_monthly

,Date,1925.T,1928.T,2413.T,2502.T,2503.T,2802.T,2914.T,3382.T,3402.T,3407.T,4063.T,4188.T,4452.T,4502.T,4503.T,4507.T,4519.T,4523.T,4528.T,4543.T,4568.T,4578.T,4661.T,4689.T,4901.T,4911.T,5020.T,5108.T,5401.T,5713.T,5802.T,6098.T,6178.T,6273.T,6301.T,6326.T,6367.T,6501.T,6502.T,6503.T,6586.T,6594.T,6645.T,6702.T,6752.T,6758.T,6861.T,6869.T,6902.T,6920.T,6954.T,6971.T,6981.T,7011.T,7201.T,7203.T,7267.T,7269.T,7270.T,7309.T,7733.T,7741.T,7751.T,7832.T,7974.T,8001.T,8002.T,8031.T,8035.T,8053.T,8058.T,8113.T,8267.T,8306.T,8308.T,8309.T,8316.T,8411.T,8591.T,8604.T,8630.T,8697.T,8725.T,8750.T,8766.T,8801.T,8802.T,8830.T,9020.T,9021.T,9022.T,9101.T,9202.T,9432.T,9433.T,9434.T,9735.T,9843.T,9983.T,9984.T,TOPIX100.T,1475.T
0,2013-11-01,1990.0,1413.0,668.25,2804.0,1578.0,1460.0,3460.0,3770.0,724.000000,808.000000,5920.0,476.000000,3365.0,4975.0,1214.000000,2252.0,815.000000,4000.0,1552.0,1337.50,626.666687,2992.0,3772.50,494.000000,2797.0,1752.0,533.000000,3750.0,3320.000000,2726.0,1597.0,NaN,NaN,24660.0,2129.0,1751.0,6500.0,3775.0,4420.0,1183.0,2555.0,2472.50,4215.0,4770.0,1175.000000,1867.0,10287.50,3330.0,1282.50,239.75,17240.0,5420.0,978.888916,6570.0,936.000000,1276.000000,1443.333374,2630.0,2892.0,9100.0,861.25,2771.0,3410.0,2081.0,1313.0,1292.0,743.000000,1419.0,5550.0,1267.0,2015.0,2160.000000,1378.0,659.000000,508.000000,5030.0,5070.0,2150.0,1865.0,811.000000,2839.0,1367.5,2757.0,1605.0,1133.333374,3475.0,2844.0,4855.0,8390.0,4480.0,2464.0,1056.666626,2070.0,51.400002,2143.333252,NaN,6320.0,4745.0,12933.333008,4145.0,NaN,NaN
1,2013-12-01,2035.0,1470.0,658.75,2964.0,1513.0,1522.0,3420.0,4180.0,728.000000,824.000000,6140.0,486.000000,3310.0,4825.0,1246.000000,2280.0,775.000000,4075.0,1842.0,1267.50,641.000000,3040.0,3792.50,585.000000,2981.0,1691.0,541.000000,3980.0,3520.000000,2754.0,1754.0,NaN,NaN,26500.0,2137.0,1739.0,6550.0,3980.0,4420.0,1320.0,2760.0,2575.00,4645.0,5440.0,1224.000000,1826.0,11250.00,3105.0,1387.50,254.50,19250.0,5250.0,1037.777832,6510.0,884.000000,1284.000000,1443.333374,2828.0,3015.0,9030.0,832.50,2922.0,3330.0,2334.0,1401.0,1299.0,756.000000,1465.0,5760.0,1321.0,2017.0,2000.000000,1425.0,694.000000,536.000000,5540.0,5420.0,2280.0,1847.0,809.000000,2924.0,1494.5,2822.0,1757.0,1171.666626,3785.0,3145.0,5230.0,8380.0,4560.0,2476.0,1120.000000,2100.0,56.599998,2156.666748,NaN,6340.0,4985.0,14466.666992,4600.0,NaN,NaN
2,2014-01-01,1964.0,1434.0,756.25,2813.0,1406.0,1454.0,3197.0,4102.0,677.000000,785.000000,5763.0,441.000000,3275.0,4793.0,1279.599976,2119.0,781.333313,3943.0,1786.0,1208.75,574.333313,3146.0,3898.75,584.000000,3027.0,1637.0,497.000000,3734.0,3150.000000,2690.0,1627.0,NaN,NaN,26025.0,2148.0,1598.0,5956.0,3960.0,4320.0,1178.0,2695.0,2881.25,4090.0,5780.0,1184.000000,1626.0,10627.50,2845.0,1334.75,274.75,16770.0,4651.0,1067.111084,6700.0,885.000000,1184.400024,1297.666626,2692.0,2847.0,9130.0,762.50,2843.0,3006.0,2330.0,1215.0,1265.0,723.000000,1385.0,5317.0,1290.0,1903.0,1873.333374,1289.0,623.000000,546.000000,4950.0,4824.0,2190.0,1590.0,725.000000,2711.0,1261.0,2412.0,1559.0,1011.333313,3289.0,2545.0,4592.0,7632.0,4226.0,2264.0,1070.000000,2180.0,55.610001,1901.666626,NaN,5830.0,4985.0,12703.333008,3781.5,NaN,NaN
3,2014-02-01,1846.0,1274.0,846.25,2852.0,1385.0,1577.0,3232.0,3813.0,701.000000,723.000000,5758.0,459.000000,3495.0,4872.0,1318.800049,2200.0,866.000000,3970.0,2014.0,1083.75,584.000000,3123.0,3817.50,645.000000,2924.0,1802.0,527.000000,3667.0,2970.000000,2660.0,1556.0,NaN,NaN,25815.0,2131.0,1423.0,5837.0,4015.0,4400.0,1205.0,2830.0,3122.50,4275.0,6340.0,1276.000000,1780.0,10917.50,3035.0,1360.00,273.00,17655.0,4591.0,1076.777832,6250.0,911.000000,1167.800049,1215.666626,2731.0,2753.0,9210.0,882.50,3016.0,3162.0,2277.0,1255.5,1264.0,713.000000,1564.0,5842.0,1342.0,1947.0,1923.333374,1244.0,587.000000,530.000000,4770.0,4540.0,2090.0,1500.0,687.000000,2569.0,1207.0,2408.0,1484.0,1004.000000,3027.0,2403.0,4093.0,7942.0,4170.0,2367.0,1083.333374,2270.0,57.009998,2067.000000,NaN,5737.0,4600

#### **0.2.3.3 週次データ確認**

In [ ]:
all_df_weekly

,Date,1925.T,1928.T,2413.T,2502.T,2503.T,2802.T,2914.T,3382.T,3402.T,3407.T,4063.T,4188.T,4452.T,4502.T,4503.T,4507.T,4519.T,4523.T,4528.T,4543.T,4568.T,4578.T,4661.T,4689.T,4901.T,4911.T,5020.T,5108.T,5401.T,5713.T,5802.T,6098.T,6178.T,6273.T,6301.T,6326.T,6367.T,6501.T,6502.T,6503.T,6586.T,6594.T,6645.T,6702.T,6752.T,6758.T,6861.T,6869.T,6902.T,6920.T,6954.T,6971.T,6981.T,7011.T,7201.T,7203.T,7267.T,7269.T,7270.T,7309.T,7733.T,7741.T,7751.T,7832.T,7974.T,8001.T,8002.T,8031.T,8035.T,8053.T,8058.T,8113.T,8267.T,8306.T,8308.T,8309.T,8316.T,8411.T,8591.T,8604.T,8630.T,8697.T,8725.T,8750.T,8766.T,8801.T,8802.T,8830.T,9020.T,9021.T,9022.T,9101.T,9202.T,9432.T,9433.T,9434.T,9735.T,9843.T,9983.T,9984.T,TOPIX100.T,1475.T
0,2020-10-05,2878.0,1790.0,6870.0,3770.0,2000.0,2117.0,2002.0,3542.0,493.000000,926.400024,14135.0,613.599976,7769.0,3712.0,1500.5,5524.0,4594.0,9375.0,3320.0,4140.0,2890.5,4230.0,14990.0,767.000000,5212.0,6587.0,385.500000,3457.0,1087.5,3348.0,1206.0,4300.0,733.200012,58080.0,2436.0,1923.0,19190.0,3650.0,2840.0,1430.5,4960.0,10150.0,8120.0,14180.0,894.000000,7862.0,49520.0,10250.0,1213.75,9340.0,20440.0,6038.0,2289.333252,2316.0,381.500000,1393.400024,860.666687,4866.0,2083.0,21220.0,2231.5,12160.0,1697.0,7812.0,5813.0,2645.5,608.299988,1848.5,28430.0,1261.0,2551.5,4790.0,2804.0,430.600006,360.399994,2834.0,2999.0,1331.0,1387.5,486.000000,3762.0,2801.0,2832.0,1607.5,1589.666626,1861.5,1623.0,2981.5,6353.0,5016.0,2992.0,654.333313,2400.5,91.779999,2694.0,1197.0,9340.0,21665.0,23073.333984,6997.0,1045.469971,1684.0
1,2020-10-12,2836.0,1795.0,7140.0,3594.0,1931.0,2120.5,1996.5,3425.0,475.399994,910.099976,14175.0,612.400024,7724.0,3504.0,1488.5,5154.0,4319.0,8552.0,3243.0,3995.0,2702.5,4043.0,14905.0,758.000000,5466.0,6745.0,370.000000,3325.0,1066.0,3238.0,1136.5,4242.0,734.400024,57520.0,2415.0,1907.0,19255.0,3550.0,2750.0,1366.5,4860.0,10095.0,7600.0,13945.0,871.599976,7603.0,48950.0,10240.0,1197.25,9270.0,20080.0,6052.0,2402.000000,2202.0,371.899994,1365.800049,843.666687,4613.0,2020.5,20440.0,2085.0,12235.0,1646.0,7848.0,5792.0,2612.0,584.400024,1795.5,29015.0,1223.5,2457.0,5006.0,2838.5,419.600006,353.000000,2850.0,2952.5,1296.0,1334.0,487.200012,3817.0,2698.0,2903.5,1585.0,1606.666626,1875.0,1600.0,2884.5,6150.0,4906.0,2855.0,633.333313,2368.5,89.760002,2764.5,1215.0,9039.0,22065.0,24476.666016,7023.0,1029.699951,1657.0
2,2020-10-19,2893.0,1773.5,6910.0,3566.0,1965.5,2088.0,1993.5,3430.0,498.899994,943.500000,14445.0,606.599976,7637.0,3500.0,1483.5,5059.0,4281.0,8330.0,3194.0,3905.0,2684.0,4070.0,15255.0,722.000000,5397.0,6881.0,367.399994,3618.0,1119.0,3458.0,1175.5,4244.0,741.599976,56770.0,2486.0,1953.0,19745.0,3650.0,2864.0,1365.5,4965.0,10600.0,7880.0,13655.0,894.299988,7820.0,48730.0,10020.0,1233.75,8850.0,21540.0,6091.0,2410.000000,2370.0,387.899994,1394.599976,860.000000,4632.0,2096.5,20330.0,2027.0,11915.0,1730.5,7845.0,5503.0,2572.5,590.099976,1804.5,28450.0,1229.0,2457.5,5112.0,2714.0,425.799988,356.100006,2903.5,3002.0,1322.0,1325.0,497.799988,4011.0,2622.0,2980.5,1677.0,1629.333374,1881.5,1608.0,2955.5,5838.0,4877.0,2759.0,672.666687,2391.5,88.559998,2770.5,1200.0,8950.0,21380.0,24220.000000,7082.0,1036.439941,1664.0
3,2020-10-26,2735.5,1726.5,7027.0,3223.0,1879.0,2097.5,1970.0,3179.0,470.399994,900.400024,13890.0,586.200012,7420.0,3239.0,1435.0,4924.0,4021.0,8089.0,2972.0,3837.0,2748.0,3855.0,14585.0,725.099976,5318.0,6450.0,350.600006,3390.0,1006.5,3223.0,1145.5,3970.0,714.900024,55300.0,2341.0,1810.5,19480.0,3502.0,2629.0,1340.0,4600.0,10500.0,7500.0,12225.0,960.700012,8674.0,47250.0,9800.0,1209.50,9040.0,22050.0,5731.0,2422.333252,2234.5,366.000000,1360.599976,811.666687,4457.0,1905.5,23765.0,1989.0,11800.0,1800.5,7776.0,5698.0,2500.0,543.200012,1630.0,27900.0,1144.5,2325.5,4840.0,2663.5,410.200012,342.500000,2783.5,2881.0,1281.5,1215.5,464.500000,3891.0,2542.5,2844.5,1546.5,1553.000000,1774.5,1554.0,2787.0,5457.0,4460.0,2517.0,638.333313,2270.0,87.839996,2792.5,1212.5,8795.0,21550.0,24236.666016

# **0.0 データの確認と前処理**

## **0.1 データの欠損値確認**

### **0.1.1 monthlyの欠損値確認**

In [ ]:
# monthlyのNaNを含む列とその数を出力
for col in all_df_monthly.columns:
    nan_count = all_df_monthly[col].isnull().sum()
    if nan_count > 0:
        print(f"Column '{col}' has {nan_count} NaN values.")

Column '6098.T' has 12 NaN values.
Column '6178.T' has 25 NaN values.
Column '9434.T' has 62 NaN values.
Column 'TOPIX100.T' has 76 NaN values.
Column '1475.T' has 24 NaN values.


### **0.1.1 weeklyの欠損値確認**

In [ ]:
# WeeklyのNaNを含む列とその数を出力
for col in all_df_weekly.columns:
    nan_count = all_df_weekly[col].isnull().sum()
    if nan_count > 0:
        print(f"Column '{col}' has {nan_count} NaN values.")

In [ ]:
print(len(all_df_weekly.columns))

103


## **0.2 データの分類**

### **0.2.1 月次/週次の市場データ作成**

In [ ]:
market_df_monthly = all_df_monthly[['Date', 'TOPIX100.T', '1475.T']]
market_df_weekly = all_df_weekly[['Date', 'TOPIX100.T', '1475.T']]

In [ ]:
df_columns = all_df_monthly.columns

### **0.2.1 月次/週次の業界別データ作成**

In [ ]:
# tickerをそれに対応する企業名の確認
ticker_to_name

{'1925.T': 'DAIWA HOUSE INDUSTRY CO',
 '1928.T': 'SEKISUI HOUSE',
 '2413.T': 'M3 INC',
 '2502.T': 'ASAHI GROUP HLDGS',
 '2503.T': 'KIRIN HOLDINGS COMPANY LIMITED',
 '2802.T': 'AJINOMOTO CO INC',
 '2914.T': 'JAPAN TOBACCO INC',
 '3382.T': 'SEVEN & I HOLDINGS CO LTD',
 '3402.T': 'TORAY INDUSTRIES INC',
 '3407.T': 'ASAHI KASEI CORP',
 '4063.T': 'SHIN-ETSU CHEMICAL CO',
 '4188.T': 'MITSUBISHI CHEMICAL GROUP CORP',
 '4452.T': 'KAO CORP',
 '4502.T': 'TAKEDA PHARMACEUTICAL CO LTD',
 '4503.T': 'ASTELLAS PHARMA',
 '4507.T': 'SHIONOGI & CO',
 '4519.T': 'CHUGAI PHARMACEUTICAL CO',
 '4523.T': 'EISAI CO',
 '4528.T': 'ONO PHARMACEUTICAL CO',
 '4543.T': 'TERUMO CORP',
 '4568.T': 'DAIICHI SANKYO COMPANY LIMITED',
 '4578.T': 'OTSUKA HLDGS CO LTD',
 '4661.T': 'ORIENTAL LAND CO',
 '4689.T': 'LY CORPORATION',
 '4901.T': 'FUJIFILM HOLDINGS CORPORATION',
 '4911.T': 'SHISEIDO COMPANY LIMITED',
 '5020.T': 'ENEOS HOLDINGS INC',
 '5108.T': 'BRIDGESTONE CORP',
 '5401.T': 'NIPPON STEEL CORPORATION',
 '5713.T': 'S

In [ ]:
ticker_to_name['8035.T']

'TOKYO ELECTRON'

In [ ]:
# 建設・不動産
construction_real_estate = ['1925.T', '1928.T', '8801.T', '8802.T', '8830.T']
construction_real_estate_names = [ticker_to_name[ticker] for ticker in construction_real_estate]

# 食品・飲料
food_beverage = ['2502.T', '2503.T', '2802.T', '2914.T']
food_beverage_names = [ticker_to_name[ticker] for ticker in food_beverage]

# 小売
retail = ['3382.T', '8267.T', '9843.T', '9983.T']
retail_names = [ticker_to_name[ticker] for ticker in retail]

# 化学・製薬
chemicals_pharmaceuticals = ['3402.T', '3407.T', '4063.T', '4188.T', '4452.T', '4502.T', '4503.T', '4507.T', '4519.T', '4523.T', '4528.T', '4568.T', '4578.T', '4901.T', '8113.T', '4911.T']
chemicals_pharmaceuticals_names = [ticker_to_name[ticker] for ticker in chemicals_pharmaceuticals]

# 金融・保険
finance_insurance = ['8306.T', '8308.T', '8309.T', '8316.T', '8411.T', '8591.T', '8604.T', '8630.T', '8697.T', '8725.T', '8750.T', '8766.T']
finance_insurance_names = [ticker_to_name[ticker] for ticker in finance_insurance]

# 情報・通信
information_communication = ['4689.T', '9432.T', '9433.T', '9434.T', '9984.T']
information_communication_names = [ticker_to_name[ticker] for ticker in information_communication]

# 輸送・物流
transport_logistics = ['9101.T', '9202.T', '9020.T', '9021.T', '9022.T']
transport_logistics_names = [ticker_to_name[ticker] for ticker in transport_logistics]

# 資源
materials = ['5401.T', '5713.T', '5020.T', '5108.T']
materials_names = [ticker_to_name[ticker] for ticker in materials]

# 機械・電機
machinery_electronics = ['6301.T', '6326.T', '6367.T', '6501.T', '6502.T', '6503.T', '6586.T', '6594.T', '6645.T', '6702.T', '6752.T', '6758.T', '6861.T', '6902.T', '5802.T', '6869.T', '6920.T', '6971.T', '6981.T', '7011.T', '6273.T', '8035.T']
machinery_electronics_names = [ticker_to_name[ticker] for ticker in machinery_electronics]

# 自動車・輸送機器
automotive_transport_equipment = ['7201.T', '7203.T', '7267.T', '7269.T', '7270.T']
automotive_transport_equipment_names = [ticker_to_name[ticker] for ticker in automotive_transport_equipment]

# 商社
trading_companies = ['8001.T', '8002.T', '8031.T', '8053.T', '8058.T']
trading_companies_names = [ticker_to_name[ticker] for ticker in trading_companies]

# 健康・医療機器
healthcare_medical_equipment = ['4543.T', '6954.T', '7733.T', '7741.T', '7751.T', '2413.T']
healthcare_medical_equipment_names = [ticker_to_name[ticker] for ticker in healthcare_medical_equipment]

# サービス・エンターテインメント
services_entertainment = ['4661.T', '7309.T', '7832.T', '7974.T', '6098.T', '9735.T', '6178.T']
services_entertainment_names = [ticker_to_name[ticker] for ticker in services_entertainment]

# その他
markets = ['TOPIX100.T', '1475.T']
markets_names = [ticker_to_name[ticker] for ticker in markets]

In [ ]:
total_tickers = [
    construction_real_estate,
    food_beverage,
    retail,
    chemicals_pharmaceuticals,
    finance_insurance,
    information_communication,
    transport_logistics,
    materials, 
    machinery_electronics,
    automotive_transport_equipment,
    trading_companies,
    healthcare_medical_equipment,
    services_entertainment,
    markets
]

# Flatten the list of tickers
all_tickers = [ticker for sublist in total_tickers for ticker in sublist]

# Count the total number of tickers
total_count = len(all_tickers)
print(total_count)
print(all_tickers)

102
['1925.T', '1928.T', '8801.T', '8802.T', '8830.T', '2502.T', '2503.T', '2802.T', '2914.T', '3382.T', '8267.T', '9843.T', '9983.T', '3402.T', '3407.T', '4063.T', '4188.T', '4452.T', '4502.T', '4503.T', '4507.T', '4519.T', '4523.T', '4528.T', '4568.T', '4578.T', '4901.T', '8113.T', '4911.T', '8306.T', '8308.T', '8309.T', '8316.T', '8411.T', '8591.T', '8604.T', '8630.T', '8697.T', '8725.T', '8750.T', '8766.T', '4689.T', '9432.T', '9433.T', '9434.T', '9984.T', '9101.T', '9202.T', '9020.T', '9021.T', '9022.T', '5401.T', '5713.T', '5020.T', '5108.T', '6301.T', '6326.T', '6367.T', '6501.T', '6502.T', '6503.T', '6586.T', '6594.T', '6645.T', '6702.T', '6752.T', '6758.T', '6861.T', '6902.T', '5802.T', '6869.T', '6920.T', '6971.T', '6981.T', '7011.T', '6273.T', '8035.T', '7201.T', '7203.T', '7267.T', '7269.T', '7270.T', '8001.T', '8002.T', '8031.T', '8053.T', '8058.T', '4543.T', '6954.T', '7733.T', '7741.T', '7751.T', '2413.T', '4661.T', '7309.T', '7832.T', '7974.T', '6098.T', '9735.T', '6178